#  Setup stuff: Connectivity

In [ ]:
#  From,
#
#     https://github.com/KatanaGraph/katana-enterprise/blob/master/python/test/notebooks/red/analytics_remote.ipynb
#
#     This file also includes; project()



In [ ]:
import os

from katana import remote


os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging 

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


#  Actual analytics ..

In [ ]:

#  Analytics are headed here in the documentation,
#     https://docs.k9h.dev/latest/user-guides/algorithms/index.html
#
#  And include,
#
#     Path Algorithms
#        Single-Source Shortest Paths (SSSP)            - No path, bugs
#        Breadth-First Search (BFS)                     - Syntactic sugar for above, wrong use/name ?
#        K-shortest paths                               - No path, bugs
#
#     Community Detection Algorithms
#        Connected Components                           -  ??
#        Community Detection using Label Propagation    -     local
#        Louvain                                        -  ??
#        Local Clustering Coefficient                   -     local
#        Triangle Counting                              -  Not implemented
#        K-core                                         -  ??
#        K-truss                                        -     local
#
#     Centrality Algorithms
#        PageRank                                       -  No weight ?  Working to validate result/formula
#        Betweenness Centrality                         -
#
#     Similarity Algorithms
#        Jaccard Similarity                             -     local



In [ ]:

# import numpy as np
from katana import remote
from katana.remote import analytics
# from katana_enterprise.distributed import PartitionPolicy


l_cntr = 0

print("--")


In [ ]:

#  Running analytics.sssp() ..


l_source_nodeid       = 2                          #  id of node to start on (this is SJC)

# l_weight_property     = None                     #  This errors out
   #
l_weight_property     = "DISTANCE"                 #  Edge property
# l_weight_property     = "NUM_HOPS"

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
l_result.view()


In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
l_result.view()


#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge
#  3            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge
#
#  1            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge

#  Graph loaded, after running sssp() 
#     (This is not correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge         #  This data is now wrong
#  1            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge         #  This data is now wrong
#
#  3            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge         #  This data is now wrong



In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


# Running graph.project()



In [ ]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph that analytics.sssp() will not like ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result0 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code    = 'SJC'
   AND   m.restaurant_code = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result0 = my_graph.query(l_query))


In [ ]:

#  analytics.sssp() now will now excounter at least one edge without a weight
#  property. As such, it generates a NaN for that path/output.


l_source_nodeid       = 2                          #  id of node to start on (this is SJC)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)

analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
l_result.view()


In [ ]:
#  analytics.sssp() should be run against a proection, a sub-graph ..

my_graph2 = my_graph.project(edge_types=["FLIES_TO"])


l_source_nodeid       = 2                          #  id of node to start on (this is SJC)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)

analytics.sssp(
   my_graph2,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))




In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
l_result.view()
